# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import yfinance as yf
import pandas_ta
import datetime as dt
import warnings

warnings.filterwarnings('ignore')

## Importing S&P 500 from Wikipedia

In [9]:
# stocks = pd.read_csv('sp_500_stocks.csv')
# stocks

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-12-15'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()

df


[*********************100%%**********************]  503 of 503 completed


adj close       close        high         low        open  \
date       ticker                                                               
2015-12-17 A        38.179028   40.779999   41.630001   40.770000   41.549999   
           AAL      40.576706   42.459999   43.950001   42.450001   43.330002   
           AAPL     24.803656   27.245001   28.062500   27.245001   28.004999   
           ABBV     39.907444   56.380001   57.930000   56.349998   57.660000   
           ABT      38.962093   45.259998   46.220001   45.250000   46.090000   
...                       ...         ...         ...         ...         ...   
2023-12-14 YUM     131.110001  131.110001  131.910004  129.350006  131.619995   
           ZBH     118.529999  118.529999  120.330002  118.430000  119.089996   
           ZBRA    272.160004  272.160004  274.440002  247.110001  248.559998   
           ZION     45.669998   45.669998   46.150002   43.740002   43.820000   
           ZTS     200.089996  200.089996  201.919998  198.000000  198.000000   

                        volume  
date       ticker               
2015-12-17 A         2218800.0  
           AAL       7225000.0  
           AAPL    179091200.0  
           ABBV      7848900.0  
           ABT       5792200.0  
...                        ...  
2023-12-14 YUM       3238700.0  
           ZBH       2049000.0  
           ZBRA      1165900.0  
           ZION      6563800.0  
           ZTS       3044400.0  

[994308 rows x 6 columns]

## Making Our First API Call

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [15]:
symbol = 'AAPL'

aapl = yf.Ticker(symbol)
aapl_info = aapl.info
aapl_mc = aapl_info['marketCap']
appl_price = aapl_info['previousClose']



197.57

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: